
---
# **License Plate Recognition**
##**David Peterson - ECE 4424 - Spring 2022 Project - AMLPR**
---


### **Data Loading and Pre-Processing**###
1.   Mount Google Drive
2.   Load original license plate image dataset - number labels
3.   Modify original license plate image dataset structure for YOLOv3 application

**Dataset Modification Details:**  

- Dataset Train-Test Split: 80% Train | 20% Train

- File Structure in Recognition Directory:
> lp_num_img_data (original)   
> recognition_data (modified)  

- Original format of dataset:  
  - Separate train and test text files with image file paths and labels for each image
> images/imageX.jpg  
> imageX label   
> ...

- Modified format of dataset:   
  - Separate train and test text files with only image file paths  
  - Separate labels directory with text files for individual image labels

  - File Structure in recognition_data directory:
> recognition_train.txt  
> recognition_test.txt  
> images  
> labels  

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# append the directory to your python path using sys
import sys
import os
prefix = '/content/drive/MyDrive/'

# customized path to detection directory
cust_path_detection = 'ECE4424/Project/detection'
sys_path_detection = prefix + cust_path_detection
sys.path.append(sys_path_detection)

# customized path to recognition directory
cust_path_recognition = 'ECE4424/Project/recognition'
sys_path_recognition = prefix + cust_path_recognition
sys.path.append(sys_path_recognition)

print("Path to detection directory: {}".format(sys_path_detection))
print("Path to recognition directory: {}".format(sys_path_recognition))

Path to detection directory: /content/drive/MyDrive/ECE4424/Project/detection
Path to recognition directory: /content/drive/MyDrive/ECE4424/Project/recognition


In [ ]:
# import dependencies
import re

# initialize path constants
LP_NUM_IMG_DATA_DIR_PATH = sys_path_recognition + "/lp_num_img_data"  # path to original dataset directory
TRAIN_DATA_FILE_PATH = LP_NUM_IMG_DATA_DIR_PATH + "/train_data.txt"   # path to train data file
TEST_DATA_FILE_PATH = LP_NUM_IMG_DATA_DIR_PATH + "/test_data.txt"     # path to test data file

# load train and test datasets
train_data_file = open(TRAIN_DATA_FILE_PATH, "r")  # open train data file
test_data_file = open(TEST_DATA_FILE_PATH, "r")    # open test data file

# separate train data into file names and labels lists
train_data_fps = []                                                   # initialize list for train data image file paths
train_data_lbls = []                                                  # initialize list for train data labels
for idx, line in enumerate(train_data_file):                          # for each line in the train data file
    if not idx % 2:                                                       # if line index is even
        train_data_fps.append(line.strip()[1:])                               # append image file name to train data image file names list
    else:                                                                 # else
        train_data_lbls.append(line.strip())                                  # append label to train data label list as a string

# organize train data into file names:labels dictionary
train_data = {}
for idx in range(len(train_data_fps)):                                # for each train image
    train_data[train_data_fps[idx]] = train_data_lbls[idx]                # insert file name-labels key-value pair

# separate test data into file names and labels lists
test_data_fps = []                                                    # initialize list for test data image file paths
test_data_lbls = []                                                   # initialize list for test data labels
for idx, line in enumerate(test_data_file):                           # for each line in the test data file
    if not idx % 2:                                                       # if line index is even
        test_data_fps.append(line.strip()[1:])                                # append image file name to test data image file names list
    else:                                                                 # else
        test_data_lbls.append(line.strip())                                   # append label to test data label list as a string

# organize test data into file names:labels dictionary
test_data = {}
for idx in range(len(test_data_fps)):                                 # for each test image
    test_data[test_data_fps[idx]] = test_data_lbls[idx]                   # insert file name-labels key-value pair

all_data_fps = train_data_fps + test_data_fps       # initialize list to store all data file paths

all_data_fns = []                                   # initialize list to store all data file names
for fp in all_data_fps:                             # for each image file path
    all_data_fns.append(re.split("/|\.", fp)[2])        # obtain file name from current input image file path


print("Train Data: {}".format(train_data))
print("Amount of Train Data: {}".format(len(train_data)))
print("Test Data: {}".format(test_data))
print("Amount of Test Data: {}".format(len(test_data)))
print("All Data File Paths: {}".format(all_data_fps))
print("All Data File Names: {}".format(all_data_fns))
print("[STATUS] Original data loaded successfully.")

Train Data: {'/images/cars4_052.jpg': '3UDX841', '/images/cars2_021.jpg': '3MYH761', '/images/cars4_074.jpg': '5HZW063', '/images/cars4_010.jpg': '5CJB364', '/images/cars3_026.jpg': '4GQG231', '/images/cars4_085.jpg': '2PXR727', '/images/cars063.jpg': '5DHU058', '/images/cars4_061.jpg': '4BRM434', '/images/cars4_080.jpg': '4CBM944', '/images/cars4_128.jpg': '5JJV943', '/images/cars4_127.jpg': '5BSM366', '/images/cars4_065.jpg': '3SFW623', '/images/cars4_037.jpg': '4LXC614', '/images/cars060.jpg': '5HCV316', '/images/cars4_105.jpg': '5GEC237', '/images/cars4_116.jpg': '3UUJ586', '/images/cars4_124.jpg': '5GAM144', '/images/cars2_052.jpg': '4UCF349', '/images/cars4_060.jpg': '5ETA743', '/images/cars4_005.jpg': '3UXY336', '/images/cars056.jpg': '4ZTJ876', '/images/cars4_020.jpg': 'MADZBMR', '/images/cars2_035.jpg': '2VLR763', '/images/cars2_032.jpg': '4VAH764', '/images/cars4_055.jpg': '3UTA977', '/images/cars2_068.jpg': '4TIX783', '/images/cars2_042.jpg': '4VLX070', '/images/cars4_026.jp

In [ ]:
# initialize path constants
RECOGNITION_DATA_DIR_PATH = sys_path_recognition + "/recognition_data"  # path to modified dataset directory

# create recognition_train.txt - contains paths for images in train data
recognition_train_file = open(RECOGNITION_DATA_DIR_PATH + "/recognition_train.txt", "w+") # open recognition_train.txt in write mode - create if does not exist
for fp in train_data.keys():                                                              # for each file path in train data
    recognition_train_file.write(RECOGNITION_DATA_DIR_PATH + fp + "\n")                      # write the file path to recognition_train.txt on a new line
recognition_train_file.close()                                                            # close recognition_train.txt
print("[STATUS] recognition_train.txt created successfully.")

# create recognition_test.txt - contains paths for images in test data
recognition_test_file = open(RECOGNITION_DATA_DIR_PATH + "/recognition_test.txt", "w+")   # open recognition_test.txt in write mode - create if does not exist
for fp in test_data.keys():                                                               # for each file path in test data
    recognition_test_file.write(RECOGNITION_DATA_DIR_PATH + fp + "\n")                        # write the file path to recognition_test.txt on a new line
recognition_test_file.close()                                                             # close recognition_test.txt
print("[STATUS] recognition_test.txt created successfully.")

[STATUS] recognition_train.txt created successfully.
[STATUS] recognition_test.txt created successfully.


In [ ]:
# import re (built-in) - multiple delimiters
import re

# initialize path constants
RECOGNITION_LABELS_DIR_PATH = RECOGNITION_DATA_DIR_PATH + "/labels/"   # path to labels directory

# combine original train and test data image labels and file paths
all_data = {}
all_data.update(train_data)
all_data.update(test_data)

# create modified train and test data labels
for fp, label in all_data.items():                                       # for each image
    fn = re.split("/|\.", fp)[2]                                             # split image file path by '/' and '.' delimiters to obtain file name
    label_file = open(RECOGNITION_LABELS_DIR_PATH + fn + ".txt", "w+")       # open image label text file in write mode - create if does not exist
    label_file.write(label)                                                  # write the label to the image label text file
    label_file.close()                                                       # close image label text file

print("[STATUS] Label files for all data created successfully.")

[STATUS] Label files for all data created successfully.


### **Data Modification**###
1.   Obtain predicted license plate location with YOLOv3 license plate detection model
2.   Crop images utilizing YOLOv3 predicted license plate location

Citations for YOLOv3 + OpenCV Image Processing: 
- https://towardsdatascience.com/  
- https://pyimagesearch.com/




In [ ]:
# install dependencies
!pip install numpy
!pip install opencv-python

In [ ]:
# relevant file paths
CLASS_NAMES_PATH = sys_path_detection + "/classes.names"
WEIGHTS_PATH = sys_path_detection + "/detection_data/weights/darknet-yolov3-train-custom_final.weights"
YOLO_CFG_PATH = sys_path_detection + "/darknet-yolov3-test-custom.cfg"

In [ ]:
# import dependencies
import numpy as np
import cv2

# load class names files and map class names to a random color
class_names_file = open(CLASS_NAMES_PATH, "r")                              # open class names file
class_names_colors = {}                                                     # dictionary to store class name:class color key:value pairs
for line in class_names_file:                                               # for each class
    class_names_colors[line.strip()] = tuple(np.random.uniform(0, 255, 3))      # assign the class a randomly generated color

# load pre-trained YOLOv3 license plate object detector
net = cv2.dnn.readNetFromDarknet(YOLO_CFG_PATH, WEIGHTS_PATH)

# determine output layers names from YOLO architecture
layer_names = net.getLayerNames()
output_layer_names = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

print("[STATUS] YOLO model loaded successfully.")

[STATUS] YOLO model loaded successfully.


In [ ]:
# import dependencies
import time

# initialize path constants
ORIG_IMGS_DIR_PATH = LP_NUM_IMG_DATA_DIR_PATH + "/images/"  # path to original dataset images directory

proc_times = []   # list to store all detection processing times for analysis

# crop each image in train and test data
for fn in all_data_fns:                                               # for each image in train data
    curr_img = cv2.imread(ORIG_IMGS_DIR_PATH + fn + ".jpg")               # open the current image
    curr_img_w, curr_img_h = curr_img.shape[1], curr_img.shape[0]         # extract the width and height of the current image

    # execute forward pass of YOLO license plate detector with blob from the current input image
    # obtain bounding boxes, associated probabilities, and processing time
    scale = 1/255.0                                                                       # set the scale
    blob = cv2.dnn.blobFromImage(curr_img, scale, (416, 416), swapRB=True, crop=False)    # create blob from input image
    net.setInput(blob)                                                                    # set input to the network

    start_time = time.time()                                                              # start detection processing timer
    outputs = net.forward(output_layer_names)                                             # get output from the network
    end_time = time.time()                                                                # end detection processing timer
    proc_times.append(end_time - start_time)                                              # calculate detection processing time

    # process results of YOLOv3 model outputs for the current image
    boxes = []                # list to store detected bounding boxes
    confidences = []          # list to store confidences of detected bounding boxes
    class_IDs = []            # list to store class ID of detected bounding boxes
    confidence_thresh = 0.7   # confidence threshold
    nms_thresh = 0.3          # non-maxima suppression threshold

    for output in outputs:                      # for each output
        for detection in output:                    # for each detection in the current output
            scores = detection[5:]                      # obtain the scores for the current detection
            class_ID = np.argmax(scores)                # extract the class ID from the scores
            confidence = scores[class_ID]               # extract the confidence/probability from the scores

            if confidence > confidence_thresh:                  # if confidence of detection is above confidence threshold
                box = detection[0:4] * np.array([curr_img_w, 
                                                curr_img_h, 
                                                curr_img_w, 
                                                curr_img_h])        # scale the bounding box coordinates to current input image size
                cx, cy, w, h = box.astype("int")                    # extract bounding box information => center (x,y) coordinates, width, and height
                tlx, tly = int(cx - (w/2)), int(cy - (h/2))         # compute top-left (x,y) coordinates from center (x,y) coordinates

                boxes.append([tlx, tly, int(w), int(h)])            # update list of detected bounding boxes
                confidences.append(float(confidence))               # update list of confidences
                class_IDs.append(class_ID)                          # update list of class IDs

    # apply non-maxima supprssion (NMS), eliminate weak + overlapping bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_thresh, nms_thresh)

    # save the current input image (not cropped) with no annotations
    cv2.imwrite(RECOGNITION_DATA_DIR_PATH + "/images_ncropped/" + fn + "_ncropped.jpg", curr_img)

    # annotate detections on current input image
    if len(idxs):                                                                                         # if there is atleast 1 license plate detection
        for i in idxs.flatten():                                                                              # for each kept index
            x, y = boxes[i][0], boxes[i][1]                                                                       # extract bounding box top-left (x,y) coordinates
            w, h = boxes[i][2], boxes[i][3]                                                                       # extract bounding box width and height
            
            # save the current input image cropped to only the license plate
            wbuf = 0   # width cropping buffer to ensure license plate characters are not cut off
            hbuf = 2   # height cropping buffer to ensure license plate characters are not cut off
            curr_img_cropped = curr_img[y-hbuf:y+h+hbuf, x-wbuf:x+w+wbuf]   # crop the current input image to the detected bounding box
            cv2.imwrite(RECOGNITION_DATA_DIR_PATH + "/images_cropped/" + fn + "_cropped.jpg", curr_img_cropped)

            bb_color = list(class_names_colors.values())[class_IDs[i]]                                            # determine color of bounding box for current class
            cv2.rectangle(curr_img, (x,y), (x+w, y+h), bb_color, 2)                                               # draw rectangle on current input image
            text = "{}: {:.4f}".format(list(class_names_colors.keys())[class_IDs[i]], confidences[i])             # set text for bounding box annotation
            cv2.putText(curr_img, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bb_color, 2)                     # draw text on current input image

    # save the current input image (not cropped) with annotations
    cv2.imwrite(RECOGNITION_DATA_DIR_PATH + "/images_ncropped/" + fn + "_ncropped_ann.jpg", curr_img)

    # print(fn + " processing and cropping completed.")

print("[STATUS] Average Image Detection Processing Time: {} seconds".format(round(sum(proc_times)/len(proc_times), 5)))   # output processing analysis
print("[STATUS] All images in train and test data cropped successfully.")

[STATUS] Average Image Detection Processing Time: 2.5961 seconds
[STATUS] All images in train and test data cropped successfully.


### **Image Processing**###
1.   Load Cropped License Plates
2.   Resizing
3.   Grayscale Conversion
4.   Noise Suppression
5.   Image Thresholding (Binarization)
6.   Image Dilation

In [ ]:
# import dependencies
import cv2
import re

# initialize path constants
CROPPED_IMGS_DIR_PATH = RECOGNITION_DATA_DIR_PATH + "/images_cropped/"    # path to cropped license plate image dataset

# process each image in train and test data
for fn in all_data_fns:  
    curr_img = cv2.imread(CROPPED_IMGS_DIR_PATH + fn +  "_cropped.jpg")  # open current image

    # resize current image to have more pixels/detail
    # no desired size
    # scale width and height by factor of 2
    # bicubic interpolation over 4×4 pixel neighborhood
    resize_curr_img = cv2.resize(curr_img, None, fx=4, fy=4, interpolation = cv2.INTER_CUBIC)

    # apply colorspace conversion to grayscale for image thresholding
    gray_curr_img = cv2.cvtColor(resize_curr_img, cv2.COLOR_BGR2GRAY)
    
    # apply weighted gaussian blur filter - remove image noise
    # 5x5 kernel size - size directly proportional to amount of blur
    # sigma set to 0 - automatically compute sigma based on kernel size
    gauss_curr_img = cv2.GaussianBlur(gray_curr_img, (5, 5), 0)

    # apply image thresholding (binarization - convert all pixels to black or white) 
    # 0 for threshold value - OTSU threshold used instead
    # 255 (white) is new pixel value if current pixel value less than OTSU threshold
    bin_curr_img = cv2.threshold(gauss_curr_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # apply image dilation to remove image noise after binarization
    # 3x3 kernel size - size directly proportional to amount of dilation
    kernel = np.ones((3, 3), np.uint8)
    dil_curr_img = cv2.dilate(bin_curr_img, kernel) 

    # save the current processed image 
    cv2.imwrite(RECOGNITION_DATA_DIR_PATH + "/images_processed/" + fn + "_processed.jpg", dil_curr_img)

print("[STATUS] All images in train and test data processed successfully.")

[STATUS] All images in train and test data processed successfully.


### **Optical Character Recognition (OCR)**###
1.   Train Custom OCR Model on License Plate Number Font (Penitentiary Gothic Fill)
2.   Input Processed License Plate Images to PyTesseract OCR Engine with Trained Custom OCR Model
3.   Evaluate Accuracy of PyTesseract OCR Engine with Trained Custom OCR Model
4.   Perform Custom OCR Model Tuning
5.   Repeat 1-4


In [ ]:
# install dependencies
!sudo apt install tesseract-ocr
!pip install pytesseract

In [ ]:
# import dependencies
import pytesseract
import cv2

# initialize path constants
PROCESSED_IMGS_DIR_PATH = RECOGNITION_DATA_DIR_PATH + "/images_processed/"  # path to cropped license plate image dataset
RECOGNITION_LABELS_DIR_PATH = RECOGNITION_DATA_DIR_PATH + "/labels/"        # path to labels directory

print("Printing Actual | Predicted License Plate Numbers...")

# recognition test over entire dataset
all_data_fns.sort()
for fn in all_data_fns:  
    curr_img = cv2.imread(PROCESSED_IMGS_DIR_PATH + fn + "_processed.jpg")  # open current image
    curr_img_label = open(RECOGNITION_LABELS_DIR_PATH + fn + ".txt").readline()

    options = '--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    curr_img_prediction = pytesseract.image_to_string(curr_img, config=options)

    print("Filename: {} => {} | {}".format(fn, curr_img_label, curr_img_prediction))

Printing Actual | Predicted License Plate Numbers...
Filename: cars026 => 4YTD483 | LYTHL83I

Filename: cars027 => 5AZJ257 | DAZI257

Filename: cars028 => 5AHH512 | I AMHE 1

Filename: cars029 => 4TAZ530 | FTA25 30:4

Filename: cars030 => 4BYZ014 | PLEYZ91L |

Filename: cars031 => 4MCB318 | LMCB2]P

Filename: cars032 => 5FVL554 | PSF VL 554

Filename: cars033 => 3VBL777 | SUBLI77.

Filename: cars036 => 5AOG200 | SA06200

Filename: cars039 => 1MLN989 | 19].

Filename: cars040 => 6F55687 | 6F55697

Filename: cars042 => 4FJX296 | ! m im S|

Filename: cars045 => 4RFV131 | LLRFV1 31)

Filename: cars051 => 5GRD613 | 'SGRD61 3]

Filename: cars052 => 5EDP688 | I S5EDP 688 |

Filename: cars053 => 3J66282 | 1366282

Filename: cars055 => 4YTN357 | LLYTN35 0,

Filename: cars056 => 4ZTJ876 | L727 J876

Filename: cars057 => 5GGG842 | a SGGGRL.

Filename: cars058 => 4ZNS935 | 1.2NS935]

Filename: cars060 => 5HCV316 | SHCV316.

Filename: cars061 => 6K75488 | 6K 75L89

Filename: c